In [1]:
import pandas as pd
from datetime import datetime
import numpy as np

In [2]:
def combine4546(date, filtered_TX):
    x = filtered_TX[filtered_TX.Date == date].iloc[0:2]
    x.Open.iloc[1] = x.Open.iloc[0]
    x.High.iloc[1] = x.High.max()
    x.Low.iloc[1] = x.Low.min()
    x.TotalVolume.iloc[1] = x.TotalVolume.sum()
    return x.iloc[1]

In [3]:
def ETL(origin='TXF1.csv', start='2019-01-01', output='TXF1_2001_2019.csv'):
    TX = pd.read_csv(origin)

    TX['datetime'] = [datetime.strptime(i, "%Y/%m/%d %H:%M:%S") for i in TX['Date'] +' ' + TX['Time']]
    TX['hour'] = [int(i[0:2]) for i in TX['Time']]
    filtered_TX = TX[(TX['hour'] >= 8) & (TX['hour'] <= 13)]
    filtered_TX = filtered_TX.reset_index(drop=True)

    Date = filtered_TX.Date.unique()
    Time = filtered_TX.Time.unique()

    Time_0845 = np.array(filtered_TX[filtered_TX.Time == '08:45:00'].Date)

    df = pd.DataFrame()
    for i in Time_0845:
        df = df.append(combine4546(date=i, filtered_TX=filtered_TX))

    for i in Time_0845:
        index = filtered_TX[(filtered_TX.Date==i) & (filtered_TX.Time == '08:45:00')].index[0]
        filtered_TX = filtered_TX.drop(axis=1, index=index)
        filtered_TX = filtered_TX.drop(axis=1, index=index+1)

    filtered_TX = filtered_TX.append(df, ignore_index=True, sort=True).sort_values('datetime')
    filtered_TX = filtered_TX.reset_index(drop = True)
    Time_1346 = np.array(filtered_TX[filtered_TX.Time == '13:46:00'].Date)
    for i in Time_1346:
        filtered_TX[filtered_TX.Date == i].iloc[-2].Close = filtered_TX[filtered_TX.Date == i].iloc[-1].Close
        filtered_TX[filtered_TX.Date == i].iloc[-2].TotalVolume += filtered_TX[filtered_TX.Date == i].iloc[-1].TotalVolume
        index = filtered_TX[(filtered_TX.Date==i) & (filtered_TX.Time == '13:46:00')].index[0]
        filtered_TX = filtered_TX.drop(axis=1, index=index)
    filtered_TX = filtered_TX.reset_index(drop = True)

    filtered_TX = filtered_TX[filtered_TX.datetime >= datetime.strptime(start,'%Y-%m-%d')]
    Date = filtered_TX.Date.unique()
    Time = filtered_TX.Time.unique()
    adjusted_TX = pd.DataFrame({"Date": np.array([list(Date)] * len(Time)).T.flatten(), 
                               "Time": list(Time) * len(Date)})
    adjusted_TX = adjusted_TX.merge(filtered_TX, how='left', on=['Date', 'Time'])

    adjusted_TX['Close'] = adjusted_TX.Close.ffill()
    naindex = list(adjusted_TX[adjusted_TX.High.isna()].index)
    adjusted_TX['High'].iloc[naindex] = adjusted_TX.Close.iloc[naindex]
    adjusted_TX['Low'].iloc[naindex] = adjusted_TX.Close.iloc[naindex]
    adjusted_TX['Open'].iloc[naindex] = adjusted_TX.Close.iloc[naindex]
    adjusted_TX['TotalVolume'] = adjusted_TX.TotalVolume.fillna(0)
    adjusted_TX['datetime'] = [datetime.strptime(i, "%Y/%m/%d %H:%M:%S") for i in adjusted_TX['Date'] +' ' + adjusted_TX['Time']]
    adjusted_TX['hour'] = [int(i[0:2]) for i in adjusted_TX['Time']]
    adjusted_TX = adjusted_TX.sort_values('datetime')
    adjusted_TX.to_csv(output, index=False)

In [6]:
ETL(origin='TXF1_201906.txt', start='2019-01-01', output='TXF1_201906_test.csv')

/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:190: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [21]:
TX = pd.read_csv('TXF1.csv')

In [22]:
TX['datetime'] = [datetime.strptime(i, "%Y/%m/%d %H:%M:%S") for i in TX['Date'] +' ' + TX['Time']]
TX['hour'] = [int(i[0:2]) for i in TX['Time']]
filtered_TX = TX[(TX['hour'] >= 8) & (TX['hour'] <= 13)]
filtered_TX = filtered_TX.reset_index(drop=True)

In [23]:
Date = filtered_TX.Date.unique()
Time = filtered_TX.Time.unique()

In [24]:
Time_0845 = np.array(filtered_TX[filtered_TX.Time == '08:45:00'].Date)

In [39]:
df = pd.DataFrame()
for i in Time_0845:
    df = df.append(combine4546(date=i, filtered_TX=filtered_TX))

In [40]:
for i in Time_0845:
    index = filtered_TX[(filtered_TX.Date==i) & (filtered_TX.Time == '08:45:00')].index[0]
    filtered_TX = filtered_TX.drop(axis=1, index=index)
    filtered_TX = filtered_TX.drop(axis=1, index=index+1)

In [41]:
filtered_TX = filtered_TX.append(df, ignore_index=True, sort=True).sort_values('datetime')

In [42]:
filtered_TX = filtered_TX.reset_index(drop = True)

In [43]:
Time_1346 = np.array(filtered_TX[filtered_TX.Time == '13:46:00'].Date)
for i in Time_1346:
    filtered_TX[filtered_TX.Date == i].iloc[-2].Close = filtered_TX[filtered_TX.Date == i].iloc[-1].Close
    filtered_TX[filtered_TX.Date == i].iloc[-2].TotalVolume += filtered_TX[filtered_TX.Date == i].iloc[-1].TotalVolume
    index = filtered_TX[(filtered_TX.Date==i) & (filtered_TX.Time == '13:46:00')].index[0]
    filtered_TX = filtered_TX.drop(axis=1, index=index)
filtered_TX = filtered_TX.reset_index(drop = True)

In [44]:
filtered_TX = filtered_TX[filtered_TX.datetime >= datetime.strptime('2001-01-01','%Y-%m-%d')]
Date = filtered_TX.Date.unique()
Time = filtered_TX.Time.unique()

In [45]:
adjusted_TX = pd.DataFrame({"Date": np.array([list(Date)] * len(Time)).T.flatten(), 
                           "Time": list(Time) * len(Date)})

In [46]:
adjusted_TX = adjusted_TX.merge(filtered_TX, how='left', on=['Date', 'Time'])

/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:190: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [50]:
adjusted_TX['Close'] = adjusted_TX.Close.ffill()
naindex = list(adjusted_TX[adjusted_TX.High.isna()].index)
adjusted_TX['High'].iloc[naindex] = adjusted_TX.Close.iloc[naindex]
adjusted_TX['Low'].iloc[naindex] = adjusted_TX.Close.iloc[naindex]
adjusted_TX['Open'].iloc[naindex] = adjusted_TX.Close.iloc[naindex]
adjusted_TX['TotalVolume'] = adjusted_TX.TotalVolume.fillna(0)
adjusted_TX['datetime'] = [datetime.strptime(i, "%Y/%m/%d %H:%M:%S") for i in adjusted_TX['Date'] +' ' + adjusted_TX['Time']]
adjusted_TX['hour'] = [int(i[0:2]) for i in adjusted_TX['Time']]


In [64]:
len(Date)

97

In [65]:
adjusted_TX.shape

(29100, 9)

In [66]:
adjusted_TX.to_csv('TXF1_2019.csv', index=False)

In [19]:
adjusted_TX.to_csv('TXF1_2017_2019.csv', index=False)

In [52]:
adjusted_TX.to_csv('TXF1_2001_2019.csv', index=False)

In [41]:
ETL(origin='/Users/benjamin/Downloads/TXF1-20191007.txt', start='2001-01-01', output='TXF1_201910.csv')

/usr/local/lib/python3.6/site-packages/pandas/core/indexing.py:189: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy



In [4]:
def thunder_attack_earn(TX, drop_point=20, stop_loss=-20, speed=5, stop_win=10, stop_time='13:44:00'):
    tx_day = TX.groupby('date').agg({'Open':'first', 'Close':'last', 'Low':'min', 'High':'max'}).reset_index()
    tx_day['day_open'] = tx_day['Open']
    TX = TX.merge(tx_day[['date', 'day_open']], on='date')
    TX['long'] = TX['day_open'] - drop_point
    TX['profit'] = TX['Close'] - TX['long']

    TX['dealed'] = TX['long'] >= TX['Low']
    dealed = TX.groupby('date').agg({'dealed':'sum'}).reset_index()
    dealed['dealed'] = dealed['dealed'] > 0
    TX = TX.drop(columns='dealed')
    TX = TX.merge(dealed, on='date')

    TX['profit_dealed'] = TX['dealed'] * TX['profit']

    TX['speed'] = None
    TX.loc[4:,'speed'] = TX['Close'].values[4:] - TX['Close'].values[:-4]

    TX['stop'] = False
    TX.loc[TX['profit_dealed'] <= stop_loss, 'stop'] = True
    TX.loc[(TX['speed'] <= speed) & (TX['profit_dealed'] >= stop_win), 'stop'] = True
    TX.loc[(TX.Time == stop_time), 'stop'] = True

    TX['got'] = TX['long'] >= TX['Low']
    start_time =TX[TX.got].groupby('date').agg({'datetime': 'first'}).\
    reset_index().rename(columns={'datetime': 'start_time'})
    TX = TX.merge(start_time, how='left', on='date')
    earn = TX[TX.stop & (TX.datetime >= TX.start_time)].groupby('date').\
    agg({'datetime': 'first', 'profit':'first'}).reset_index()

    earn.loc[earn.profit < stop_loss, 'profit'] = stop_loss
    earn['money'] = earn.profit * 50 - 62
    return earn
#     return {'profit': earn.money.sum(), 'deals':len(earn), 'wins':len(earn[earn['money']>0])}


def thunder_attack(TX, drop_point=20, stop_loss=-20, speed=5, stop_win=10, stop_time='13:25:00'):
    earn = thunder_attack_earn(TX, drop_point, stop_loss, speed, stop_win, stop_time)
    return {'profit': earn.money.sum(), 'deals':len(earn), 'wins':len(earn[earn['money']>0])}

def thunder_attack_combo(TX, drop_point=20, stop_loss=-20, speed=5, stop_win=10, stop_time='13:25:00'):
    earn = thunder_attack_earn(TX, drop_point, stop_loss, speed, stop_win, stop_time)
    accumulate = 0
    for i in earn.money.values:
        times = accumulate // 30000 + 1 if accumulate // 30000 > -1 else 1
        times = times if times < 100 else 100
        accumulate += (i * times)
    return {'profit': accumulate, 'deals':len(earn), 'wins':len(earn[earn['money']>0])}


def thunder_attack_combo_earn(TX, drop_point=20, stop_loss=-20, speed=5, stop_win=10, stop_time='13:25:00'):
    earn = thunder_attack_earn(TX, drop_point, stop_loss, speed, stop_win, stop_time)
    accumulate = 0
    aaa = []
    adjust = []
    times = 1
    for i in earn.money.values:
        times = accumulate // 30000 + 1 if accumulate // 30000 > -1 else 1
        times = times if times < 100 else 100
        adjust.append(i * times)
        accumulate += (i * times)
        aaa.append(accumulate)
    earn['accumulate'] = aaa
    earn['adjust'] = adjust
    return earn



In [204]:
TX = pd.read_csv('TXF1_201910.csv')

In [205]:
f = lambda x:x[:10]
TX['date'] = TX.datetime.apply(f)

In [206]:
TX = TX[TX.datetime < '2019-10-07']

In [231]:
f = lambda x:x[:4]
TX['year'] = TX.datetime.apply(f)

In [245]:
earn = thunder_attack_earn(TX[(TX.datetime >= '2001-02-08') & (TX.year == '2001')], 
                           drop_point=10, stop_loss=-5, speed=5, stop_win=20, stop_time='13:25:00')

In [239]:
earn['year'] = earn.date.apply(f)

In [246]:
earn.money.sum()

-3228.0

In [210]:
earn[earn.date <= '2001-02-08'].money.cumsum()

0     1088.0
1      526.0
2      -36.0
3     -598.0
4     1390.0
5     2728.0
6     2166.0
7     1604.0
8     1042.0
9      480.0
10     -82.0
11    -644.0
12   -1206.0
13   -1768.0
14   -2330.0
15   -2892.0
16   -3454.0
17   -4016.0
18   -1828.0
Name: money, dtype: float64

In [10]:
earn['accumulate'] = earn.money.cumsum()

In [247]:
earn = thunder_attack_earn(TX, drop_point=10, stop_loss=-5, speed=5, stop_win=20, stop_time='13:25:00')
earn['accumulate'] = earn.money.cumsum()

In [252]:
earn

,date,datetime,profit,money,accumulate,year
0,2001-01-02,2001-01-02 09:45:00,23.0,1088.0,1088.0,2001
1,2001-01-04,2001-01-04 08:46:00,-5.0,-312.0,776.0,2001
2,2001-01-05,2001-01-05 08:46:00,-5.0,-312.0,464.0,2001
3,2001-01-08,2001-01-08 08:46:00,-5.0,-312.0,152.0,2001
4,2001-01-10,2001-01-10 09:16:00,-5.0,-312.0,-160.0,2001
5,2001-01-11,2001-01-11 09:54:00,28.0,1338.0,1178.0,2001
6,2001-01-12,2001-01-12 09:03:00,-5.0,-312.0,866.0,2001
7,2001-01-15,2001-01-15 09:01:00,-5.0,-312.0,554.0,2001
8,2001-01-16,2001-01-16 08:48:00,-5.0,-312.0,242.0,2001
9,2001-01-17,2001-01-17 08:46:00,-5.0,-312.0,-70.0,2001


In [253]:
f = lambda x: x[:4]
earn['year'] = earn.date.apply(f)
earn_year = earn.groupby('year').agg({'money':'sum'}).reset_index()

In [258]:
data = [
    go.Bar(x = earn_year.year, y = earn_year.money)]
layout = go.Layout(
        xaxis=dict(title="Year"),
        yaxis=dict(title="Profit (NTD)"),
        font=dict(
        family="Courier New, monospace",
        size=18,
        color="#7f7f7f")
)
figure = go.Figure(data=data, layout=layout)
iplot(figure, filename='year')
plot(figure, filename='return_years.html')

'file:///Users/benjamin/notebook/return_years.html'

In [255]:
data = [
    go.Scatter(x = earn.date, y = earn.accumulate)]
layout = go.Layout(
        xaxis=dict(title="Date"),
        yaxis=dict(title="Profit (NTD)"),
        font=dict(
        family="Courier New, monospace",
        size=18,
        color="#7f7f7f")
)
figure = go.Figure(data=data, layout=layout)
iplot(figure, filename='accumulate')
plot(figure, filename='accumulate_better.html')

'file:///Users/benjamin/notebook/accumulate_better.html'

In [242]:
earnq = thunder_attack_combo_earn(TX, drop_point=10, stop_loss=-5, speed=5, stop_win=20, stop_time='13:25:00')

In [249]:
earnq['accumulate'] = earnq.adjust.cumsum()

In [250]:
earnq

,date,datetime,profit,money,accumulate,adjust
0,2001-01-02,2001-01-02 09:45:00,23.0,1088.0,1088.0,1088.0
1,2001-01-04,2001-01-04 08:46:00,-5.0,-312.0,776.0,-312.0
2,2001-01-05,2001-01-05 08:46:00,-5.0,-312.0,464.0,-312.0
3,2001-01-08,2001-01-08 08:46:00,-5.0,-312.0,152.0,-312.0
4,2001-01-10,2001-01-10 09:16:00,-5.0,-312.0,-160.0,-312.0
5,2001-01-11,2001-01-11 09:54:00,28.0,1338.0,1178.0,1338.0
6,2001-01-12,2001-01-12 09:03:00,-5.0,-312.0,866.0,-312.0
7,2001-01-15,2001-01-15 09:01:00,-5.0,-312.0,554.0,-312.0
8,2001-01-16,2001-01-16 08:48:00,-5.0,-312.0,242.0,-312.0
9,2001-01-17,2001-01-17 08:46:00,-5.0,-312.0,-70.0,-312.0


In [256]:
data = [
    go.Scatter(x = earnq.date, y = earnq.accumulate)]
layout = go.Layout(
        xaxis=dict(title="Date"),
        yaxis=dict(title="Profit (NTD)"),
        font=dict(
        family="Courier New, monospace",
        size=18,
        color="#7f7f7f")
)
figure = go.Figure(data=data, layout=layout)
iplot(figure, filename='accumulate')
plot(figure, filename='accumulate_combo_better.html')

'file:///Users/benjamin/notebook/accumulate_combo_better.html'

In [257]:
data = [
    go.Scatter(x = earnq.date, y = earnq.adjust / earnq.money)]
layout = go.Layout(
        xaxis=dict(title="Date"),
        yaxis=dict(title="口數(Lot)"),
        font=dict(
        family="Courier New, monospace",
        size=18,
        color="#7f7f7f")
)
figure = go.Figure(data=data, layout=layout)
iplot(figure, filename='lot')
plot(figure, filename='lot_better.html')

'file:///Users/benjamin/notebook/lot_better.html'

In [213]:
losses = []
loss = 0
dates = []
date=""
for i in earn.values:
    if i[2] < 0:
        loss += 1
        date = i[0]
    else:
        dates.append(date)
        losses.append(loss)
        loss = 0

In [214]:
loss_streak = pd.DataFrame({'date': dates, "loss":losses})

In [5]:
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)


In [215]:
f = lambda x: x[:4]
loss_streak['year'] = loss_streak.date.apply(f)

In [216]:
loss_streak_year = loss_streak.groupby(['year', 'loss']).agg({'date':'count'}).reset_index()
loss_streak_year = loss_streak_year[loss_streak_year.loss>1].reset_index(drop=True)

In [16]:
data = [go.Heatmap(
    x=loss_streak_year.loss,
    y=loss_streak_year.year,
    z=loss_streak_year.date,
    colorbar=dict(title='連輸N次的情況經歷幾次')
                  )]
layout = go.Layout( 
    title="連輸 Loss streak",
    xaxis=dict(title="連輸次數"),
    yaxis=dict(title="year"),
    font=dict(
        family="Courier New, monospace",
        size=18,
        color="#7f7f7f")
)
figure = go.Figure(data=data, layout=layout)
iplot(figure, filename='loss streak')
plot(figure, filename='loss_streak.html')

'file:///Users/benjamin/notebook/loss_streak.html'

In [16]:
f = lambda x: x[:4]
TX['year'] = TX.date.apply(f)

In [85]:
'2001-11-12'[5:7]

'11'

In [217]:
f = lambda x: x[5:7]
loss_streak['month'] = loss_streak.date.apply(f)

In [18]:
loss_streak_month = loss_streak[loss_streak.loss>=7].groupby(['month']).agg({'date':'count'}).reset_index().sort_values(['month'])

In [21]:
loss_streak_month

,month,date
0,01,3
1,02,4
2,03,9
3,04,1
4,05,6
5,06,6
6,07,2
7,08,3
8,09,7
9,10,5


In [94]:
loss_streak[loss_streak.loss>=5]

,date,loss,year,month
3,2001-02-08,12,2001,02
5,2001-03-05,12,2001,03
6,2001-03-16,6,2001,03
10,2001-04-24,11,2001,04
11,2001-05-07,6,2001,05
13,2001-05-23,7,2001,05
15,2001-06-15,12,2001,06
19,2001-07-20,5,2001,07
23,2001-08-13,5,2001,08
30,2001-09-26,12,2001,09


In [218]:
loss_streak[loss_streak.month.isin(['01', '02']) & (loss_streak.loss>=10)]

,date,loss,year,month
3,2001-02-07,12,2001,02
48,2002-01-10,12,2002,01
935,2013-02-01,18,2013,02
1072,2015-01-20,10,2015,01


In [95]:
TX_2013 = TX[(TX.year == '2002') & (TX.datetime > '2002-03-31')]

In [109]:
metrix = []
for drop_point in range(10, 110, 10):
    for stop_loss in range(-10, -45, -5):
        for stop_win in range(5, 25, 5):
            temp = thunder_attack(TX,
                                  drop_point=drop_point, stop_loss=stop_loss, stop_win=stop_win, speed=3)
            temp['drop_point'] = drop_point
            temp['stop_loss'] = stop_loss
            temp['stop_win'] = stop_win
            metrix.append(temp)

In [110]:
metrix = pd.DataFrame(metrix)

In [111]:
metrix = metrix.sort_values('profit', ascending=False)

In [112]:
metrix

,deals,drop_point,profit,stop_loss,stop_win,wins
3,3987,10,285406.0,-10,20,1401
2,3988,10,273794.0,-10,15,1615
1,3988,10,235294.0,-10,10,1947
6,3987,10,217206.0,-15,15,1897
7,3986,10,209968.0,-15,20,1677
31,3255,20,205190.0,-10,20,1116
5,3987,10,185456.0,-15,10,2226
10,3987,10,182556.0,-20,15,2111
11,3986,10,175118.0,-20,20,1894
30,3255,20,174440.0,-10,15,1279


In [99]:
metrix[(metrix.profit>0) & (metrix.drop_point<=50)]

,deals,drop_point,profit,stop_loss,stop_win,wins
3,179,10,10902.0,-10,20,59
2,179,10,3402.0,-10,15,64
56,126,30,1988.0,-10,5,82
1,179,10,1702.0,-10,10,80
7,179,10,1402.0,-15,20,68


In [54]:
thunder_attack(TX[TX.year == '2002'], drop_point=40, stop_loss=-10, 
                           stop_win=20)

{'profit': -13184.0, 'deals': 132, 'wins': 35}

In [93]:
thunder_attack(TX[(TX.year == '2001') & (TX.datetime>'2001-03-31')], 
               drop_point=50, stop_loss=-15, stop_win=20)

{'profit': 8742.0, 'deals': 84, 'wins': 38}

In [102]:
earn = thunder_attack_earn(TX, drop_point=10, stop_loss=-10, stop_win=20)

In [104]:
earn[earn.date == '2010-03-01']

,date,datetime,profit,money
1976,2010-03-01,2010-03-01 10:05:00,112.0,5538.0


In [106]:
TX[TX.date == '2010-03-01']

,Date,Time,Close,High,Low,Open,TotalVolume,datetime,hour,date,year
680100,2010/3/1,08:46:00,7427.0,7430.0,7413.0,7413.0,1340.0,2010-03-01 08:46:00,8,2010-03-01,2010
680101,2010/3/1,08:47:00,7424.0,7431.0,7423.0,7428.0,690.0,2010-03-01 08:47:00,8,2010-03-01,2010
680102,2010/3/1,08:48:00,7422.0,7425.0,7419.0,7424.0,414.0,2010-03-01 08:48:00,8,2010-03-01,2010
680103,2010/3/1,08:49:00,7421.0,7423.0,7419.0,7422.0,208.0,2010-03-01 08:49:00,8,2010-03-01,2010
680104,2010/3/1,08:50:00,7410.0,7421.0,7410.0,7421.0,387.0,2010-03-01 08:50:00,8,2010-03-01,2010
680105,2010/3/1,08:51:00,7413.0,7413.0,7407.0,7412.0,353.0,2010-03-01 08:51:00,8,2010-03-01,2010
680106,2010/3/1,08:52:00,7416.0,7417.0,7412.0,7413.0,184.0,2010-03-01 08:52:00,8,2010-03-01,2010
680107,2010/3/1,08:53:00,7416.0,7417.0,7413.0,7416.0,175.0,2010-03-01 08:53:00,8,2010-03-01,2010
680108,2010/3/1,08:54:00,7419.0,7420.0,7416.0,7416.0,240.0,2010-03-01 08:54:00,8,2010-03-01,2010
680109,2010/3/1,08:55:00,7420.0,7421.0,7418.0,7419.0,158.0,2010-03-01 08:55:00,8,2010-03-01,2010


In [108]:
data = [go.Scatter(x=TX[TX.date == '2010-03-01'].datetime, y=TX[TX.date == '2010-03-01'].Close)]
iplot(data, filename='earn')

In [103]:
data = [go.Scatter(x=earn.date, y=earn.money)]
iplot(data, filename='earn')

In [100]:
thunder_attack(TX[(TX.year == '2002') & (TX.datetime>'2002-03-31')], 
               drop_point=10, stop_loss=-10, stop_win=20)

{'profit': 10902.0, 'deals': 179, 'wins': 59}

In [78]:
thunder_attack(TX[(TX.year == '2001') & (TX.datetime<='2001-02-08')], drop_point=10, stop_loss=-10, 
                           stop_win=20)

{'profit': -4016.0, 'deals': 18, 'wins': 3}

In [ ]:
thunder_attack(TX[(TX.year == '2002') & (TX.datetime<='2002-02-08')], drop_point=10, stop_loss=-10, 
                           stop_win=20)

In [48]:
thunder_attack(TX[TX.year == '2015'], drop_point=50, stop_loss=-30, 
                           stop_win=5)

{'profit': -7568.0, 'deals': 114, 'wins': 88}

In [41]:
thunder_attack(TX[TX.year == '2013'], drop_point=20, stop_loss=-40, 
                           stop_win=10)

{'profit': 14412.0, 'deals': 149, 'wins': 111}

In [159]:
metrix

,deals,drop_point,profit,stop_loss,stop_win,wins
51,148,20,20924.0,-35,20,93
55,148,20,20374.0,-40,20,95
25,193,10,18384.0,-40,10,141
47,148,20,17024.0,-30,20,89
50,148,20,17024.0,-35,15,97
54,148,20,16974.0,-40,15,99
26,193,10,14684.0,-40,15,123
53,149,20,14412.0,-40,10,111
48,149,20,14262.0,-35,5,130
17,193,10,13734.0,-30,10,134


In [203]:
earn = thunder_attack_earn(TX, drop_point=10, stop_loss=-10, stop_win=20)

NameError: name 'TX' is not defined

In [6]:
from bs4 import BeautifulSoup
import requests
import time
import pandas as pd
from datetime import datetime, timedelta
import numpy as np
import re

def get_position_link(url):
    
    '''
    This function has for role to send a request to Glassdoor and crawlers for links which have for class 'jobLink'.
    get_position_links() collects data science applications in every
    page which get_all_link() asks for
    Args:
           url: page URL
    returns: Python list:
            links: all links for job applications present a single page.
    '''
    links = []
    header = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}
    response = requests.get(url,headers=header)
    soup = BeautifulSoup(response.text, 'html.parser')

    a = soup.find_all('a', class_='jobLink')
    for i in a:
        links.append('https://www.glassdoor.com' + i.get('href'))

    return links




In [7]:
def get_all_links(num_page, url):
    
    '''
    This function is meant to collect all jobs applications and stores the data in
    a single link for later and faster access.
    Args:
            num_page: Number of pages get_position_link() function should crawlers in
            url: The URL of a single page.
    returns: Python List:
            link: all links which get_position_link() has been crawling in. Links of job applications
    '''
    link = []
    i = 1
    print('Collecting links....')
    while i <= num_page:
        try:
            url_main = url + str(i) + '.htm'
            link.append(get_position_link(url_main))
            i = i + 1
            time.sleep(0.5)
        except:
            print('No more pages found.')
    return link



In [201]:
def scrap_job_page(url):
        
    '''
    This function collects all data we are asking for and store the result in a dictionary.
    
    Args:
            url: a single url of a job application.
    
    return: Python dictionary
            dictionary returning data we asked the crawler to collect for us.
            
    
    '''
    dic = {}
    header = {
        'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}
    response = requests.get(url, headers=header)
    soup = BeautifulSoup(response.text, 'html.parser')
    body = soup.find('body')
    #     print(title)

    try:
        #         job title
        dic['job_title'] = body.find('h2', class_='noMargTop margBotXs strong').text.strip()
    except:
        dic['job_title'] = np.nan

    try:
        # company name
        dic['company_name'] = body.find('span', class_='strong ib').text.strip()
    except:
        dic['company_name'] = np.nan

    try:
        # location
        location = body.find('span', class_='subtle ib').text.strip().replace('–\xa0', '')
        dic['location'] = location
    except:
        dic['location'] = np.nan
        
    
    x = body.find('script')
    

    try:
        m = re.search('\"latitude\": \"[0-9]*\.[0-9]*', body.text)
        dic['latitude'] = float(m.group(0).replace('\"latitude\": \"', ''))
    except:
        dic['latitude'] = np.nan
        
    try:
        m = re.search('\"longitude\": \"[0-9]*\.[0-9]*', body.text)
        dic['longitude'] = float(m.group(0).replace('\"longitude\": \"', ''))
    except:
        dic['longitude'] = np.nan


    try:
        m = re.search('\"salarySource\":\"[\w]*', x.text)
        dic['salary_source'] = m.group(0).replace('\"salarySource\":\"', '')
    except:
        dic['salary_source'] = np.nan
    try:
        m = re.search('\"payLow\":[0-9]*', x.text)
        dic['salary_min'] = int(m.group(0).replace('\"payLow\":', ''))
    except:
        dic['salary_min'] = np.nan
    try:
        m = re.search('\"payHigh\":[0-9]*', x.text)
        dic['salary_max'] = int(m.group(0).replace('\"payHigh\":', ''))
    except:
        dic['salary_max'] = np.nan
    
    try:
        m = re.search('\"payMed\":[0-9]*', x.text)
        dic['salary_med'] = int(m.group(0).replace('\"payMed\":', ''))
    except:
        dic['salary_med'] = np.nan
    try:
        m = re.search('\"payPeriod\":[0-9]*', x.text)
        dic['salary_period'] = m.group(0).replace('\"payPeriod\":', '')
    except:
        dic['salary_period'] = np.nan
    
    
    try:
        # date
        date = body.find('span', class_='minor nowrap').text.strip()
        split = date.split(" ")
        if "second" in split or "seconds" in split:
            dic["date_posted"] = datetime.today().date()
        if "minute" in split or "minutes" in split:
            dic["date_posted"] = datetime.today().date()
        if "hours" in split or "hour" in split:
            dic["date_posted"] = datetime.today().date()
        if "week" in split or "weeks" in split:
            dic["date_posted"] = (datetime.today() - (timedelta(days=int(split[0]) * 7))).date()
        if "days" in split or "day" in split:
            dic["date_posted"] = (datetime.today() - timedelta(days=int(split[0]))).date()
        if "month" in date or "months" in date:
            dic["date_posted"] = (datetime.today() - (timedelta(days=int(split[0]) * 30))).date()
    except:

        dic["date_posted"] = datetime.today().date()

    #     Job description
    list_skills = []
    job_des = body.find('div', class_='jobDesc')

    for i in job_des:
        try:
            for li in i.find_all("li"):
                list_skills.append(li.text.strip())
        except:
            break
    try:

        dic['job_description'] = list_skills
    except:
        dic['job_description'] = np.nan

    return dic

In [167]:
links = get_all_links(30, 'https://www.glassdoor.com/Job/new-delhi-jobs-SRCH_IL.0,9_IC2891681.htm?srs=TAB_OVER_COMPANY_SEARCH')
flatten = [item for sublist in links for item in sublist]
remove_duplicates = list(set(flatten))


list_result = []

for page in remove_duplicates:
    try:
        list_result.append(scrap_job_page(page))
        print(page)
    except:
        pass
    time.sleep(0.5)
#Save the dictionary into a dataframe
df_glass = pd.DataFrame.from_dict(list_result)
#The program will create an Excel file named data_glassdoor in the same directory as this script 
writer = pd.ExcelWriter('data_glassdoor.xlsx', engine='openpyxl')
#Writing data into the Excel file
df_glass.to_excel(writer, index=False)
df_glass.to_excel(writer, startrow=len(df_glass) + 2, index=False)
writer.save()

In [202]:
scrap_job_page(remove_duplicates[0])

{'job_title': 'Associate Consultant',
 'company_name': 'Bristlecone',
 'location': 'New Delhi',
 'latitude': 28.6083,
 'longitude': 77.2008,
 'salary_source': nan,
 'salary_min': nan,
 'salary_max': nan,
 'salary_med': nan,
 'salary_period': '',
 'date_posted': datetime.date(2019, 11, 19),
 'job_description': []}

In [192]:
scrap_job_page(remove_duplicates[1])

{'job_title': 'Procurement Executive',
 'company_name': 'Cubical Laboratories private Limited',
 'location': 'New Delhi',
 'salary_estimated': nan,
 'salary_min': nan,
 'salary_max': nan,
 'date_posted': datetime.date(2019, 11, 19),
 'job_description': ['*',
  '*',
  'Prepare documents for supply of various materials.',
  'Check the proforma invoice as per the indent invited from the vendors with approved rates get certified and submit it for purchase order.',
  'Prepare MIS report.',
  'Followup with vendor for delivery of the material for various projects.',
  'Maintain Procurement record for internal and external audit.',
  'Prepare material requisition and material quality inspection',
  'Provident fund (PF)',
  'Electrical Engineering']}